In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(300)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

directory_out = './file/3.part'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

directory_out_2 = './file/3.vehicle'

if os.path.exists(directory_out_2):
    shutil.rmtree(directory_out_2)
os.makedirs(directory_out_2)

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

import json

# = = = = = = = = = = = = = = =

import re
import html as html_lib

# = = = = = = = = = = = = = = =

import itertools

# = = = = = = = = = = = = = = =

from tqdm import tqdm

# = = = = = = = = = = = = = = =

directory_in = './file/2.url'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    # = = = = = = = = = = = = = = =

    list_dict_correct, list_dict_correct_2, list_series_error, df_input = [], [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                                                    header=0,
                                                                                                    dtype=str).fillna('')

    df_input['No'] = df_input['No'].astype(int)

    count_remain = len(df_input)

    print(f'总数量：{count_remain}')
    print()

    # = = = = = = = = = = = = = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_dict_correct_2, list_series_error, count_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = = = = = = = = = = = = = =

        try:
            count_retry = 0
            while True:
                count_retry += 1

                try:
                    url_request = crawler_series['Url']

                    resp = requests.get(url_request,
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(5, 15))

                    if resp.status_code == 200:
                        soup = BeautifulSoup(resp.text, 'lxml')
                        html = etree.HTML(str(soup))

                        dict_ = json.loads(html.xpath('//script[@type="application/json"]/text()')[0])
                        if dict_:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            dict_src, list_image = {}, dict_['images']
            for i, image in enumerate(list_image):
                dict_src[str(i)] = f'''https:{image['main']['url'].strip()}''' if image['main']['url'].strip().startswith('//') else image['main']['url'].strip()

            # = = = = = = = = = = = = = = =

            list_vehicle, list_dict_fitment = [], dict_['fitment']
            for dict_fitment in list_dict_fitment:
                dict_fitment.pop('idx')
                list_key = dict_fitment.keys()
                list_value = [dict_fitment[key] if type(dict_fitment[key]) == list else [dict_fitment[key]] for key in list_key]
                list_value.append([f'{dict_['catalog'].title().strip()};{dict_['sku'].strip()}'])
                list_key = list(list_key) + ['JOIN_CPN']

                product_combination = itertools.product(*list_value)

                for combination in product_combination:
                    list_vehicle.append(dict(zip(list_key, combination)))

            # = = = = = = = = = = = = = = =

            subtitle = html.xpath('//div[@class="page-builder-layout-module desktop-only"]/descendant::p[@class="product-subtitle"]/text()')[0].strip() if html.xpath('//div[@class="page-builder-layout-module desktop-only"]/descendant::p[@class="product-subtitle"]/text()') else ''

            # = = = = = = = = = = = = = = =

            list_li, dict_count = html.xpath('//h2[@class="product-details-title"]/following-sibling::ul[@class="field-list"][1]/li'), {}
            for li in list_li:
                key = re.sub(r'<.*?>', '', etree.tostring(li.xpath('./span[@class="list-label"]')[0]).decode('utf-8'))
                key = re.sub(r' +', ' ', key.strip())
                key = html_lib.unescape(key)
                key = key.removesuffix(':').strip()
                if key == 'Brand':
                    continue
                elif key in dict_count:
                    dict_count[key] += 1
                else:
                    dict_count[key] = 1

            dict_detail, dict_no = {}, {}
            for i, li in enumerate(list_li):
                key = re.sub(r'<.*?>', '', etree.tostring(li.xpath('./span[@class="list-label"]')[0]).decode('utf-8'))
                key = re.sub(r' +', ' ', key.strip())
                key = html_lib.unescape(key)
                key = key.removesuffix(':').strip()
                if key == 'Brand':
                    list_src = li.xpath('./span[@class="list-value"]/div[@class="part-manufacturer"]/div[@class="manufacturer-logos"]/img/@src')
                    list_src = ['.'.join(src.split('/')[-1].split('.')[:-1]).title().strip() for src in list_src]
                    dict_detail[str(i)] = {key: ';'.join(list_src)}
                else:
                    value = re.sub(r'<.*?>', '', etree.tostring(li.xpath('./span[contains(@class, "list-value")]')[0]).decode('utf-8'))
                    value = re.sub(r' +', ' ', value.strip())
                    value = html_lib.unescape(value)
                    value = value.removesuffix(':').strip()
                    if dict_count[key] == 1:
                        dict_detail[str(i)] = {key: value}
                    else:
                        if key in dict_no:
                            dict_no[key] += 1
                        else:
                            dict_no[key] = 1
                        dict_detail[str(i)] = {f'{key}-{dict_no[key]}': value}

            # = = = = = = = = = = = = = = =

            with semaphore_correct:
                list_dict_correct.append({'No': crawler_series['No'],
                                          'Catalog': dict_['catalog'].title().strip(),
                                          'Part Number': dict_['sku'].strip(),
                                          'Title': dict_['title'].strip(),
                                          'Subtitle': subtitle,
                                          'Price': dict_['price'],
                                          'Picture': '',
                                          'Url': url_request,
                                          'Json_Src': json.dumps(dict_src),
                                          'Json_Detail': json.dumps(dict_detail),
                                          'JOIN_CPN': f'{dict_['catalog'].title().strip()};{dict_['sku'].strip()}'})

                # = = = = = = = = = = = = = = =

                list_dict_correct_2.extend(list_vehicle)

            # = = = = = = = = = = = = = = =

            crawler_status = True
        except:
            crawler_series['Request_Url'] = url_request

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = = = = = = = = = = = = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and count_remain == 1 or count_remain % 300 == 0 and count_remain != 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_request}\n''')

        with semaphore_remain:
            count_remain -= 1

    # = = = = = = = = = = = = = = =

    pool.map(crawler, df_input.iterrows())

    print('输出ing...')
    print()
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No', 'Part Number'],
                                                                 ascending=[True, True],
                                                                 ignore_index=True)
        df_correct['No'] = range(1, len(df_correct) + 1)
        df_correct.to_excel(f'''{directory_out}/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if list_dict_correct_2:
        df_correct = pd.DataFrame(list_dict_correct_2).sort_values(by=['JOIN_CPN', 'make', 'model', 'year'],
                                                                   ascending=[True, True, True, False],
                                                                   ignore_index=True)
        df_correct.to_excel(f'''{directory_out_2}/{file.removesuffix('.xlsx')}-vehicle-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = [file for file in sorted(os.listdir(directory_out)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_out}/{file}')]
for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'{directory_out}/{file}',
              f'''{directory_out}/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

list_file = [file for file in sorted(os.listdir(directory_out_2)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_out_2}/{file}')]
for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'{directory_out_2}/{file}',
              f'''{directory_out_2}/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：10

[True] - [请求1次] - [剩余9条] - [crawler_1（50.00%）] - [2024/11/11 15:38:15] - 5. https://toyota.oempartsonline.com/oem-parts/toyota-resonator-178930v010

[True] - [请求1次] - [剩余8条] - [crawler_1（50.00%）] - [2024/11/11 15:38:15] - 9. https://toyota.oempartsonline.com/oem-parts/toyota-resonator-5329560010

[True] - [请求1次] - [剩余7条] - [crawler_1（50.00%）] - [2024/11/11 15:38:15] - 7. https://toyota.oempartsonline.com/oem-parts/toyota-resonator-1789362070

[True] - [请求1次] - [剩余6条] - [crawler_1（50.00%）] - [2024/11/11 15:38:16] - 8. https://toyota.oempartsonline.com/oem-parts/toyota-resonator-1789428130

[True] - [请求1次] - [剩余5条] - [crawler_1（50.00%）] - [2024/11/11 15:38:16] - 4. https://toyota.oempartsonline.com/oem-parts/toyota-resonator-178930p100

[True] - [请求1次] - [剩余4条] - [crawler_1（50.00%）] - [2024/11/11 15:38:16] - 1. https://toyota.oempartsonline.com/oem-parts/toyota-resonator-178050p050

[True] - [请求1次] - [剩余3条] - [crawler_1（50.00%）] - [2024/11/11 15:38:16] - 10. https://toyota.oempar

Progress: 100%|███████████████████████████████| 2/2 [00:00<00:00, 546.95it/s]

Done ~



C:\Users\Administrator\Desktop\venv\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
